- ### Sample Test Queries (based on SHL task)

In [19]:
import pandas as pd
import requests

# Load your original dataset
df = pd.read_csv("shl_data.csv")
df.columns = df.columns.str.strip()

sample_queries = [
    {"query": "Account Manager", "expected": "Account Manager"},
    {"query": "Teller", "expected": "Teller with Sales"},
    {"query": "Director", "expected": "Director"},
    {"query": "Cashier", "expected": "Cashier"},
    {"query": "Administrative", "expected": "Administrative Assistant"}
]

In [21]:
def recall_at_k(expected, predicted, k=3):
    return int(expected.lower() in [p.lower() for p in predicted[:k]])

def map_at_k(expected, predicted, k=3):
    for i, p in enumerate(predicted[:k]):
        if p.lower() == expected.lower():
            return 1 / (i + 1)
    return 0

In [27]:
# Set your API endpoint here — local or deployed
API_URL = "https://shl-recommender-fp00.onrender.com/recommend"  # Render URL

recall_scores = []
map_scores = []

for item in sample_queries:
    query = item['query']
    expected = item['expected']

    try:
        response = requests.get(API_URL, params={"query": query})
        result = response.json()
        print(f"Raw API response for '{query}':", result)

        predictions = [r['assessment_name'] for r in result]

    except Exception as e:
        predictions = []
        print(f"Error for query '{query}':", e)

    recall = recall_at_k(expected, predictions)
    map3 = map_at_k(expected, predictions)

    recall_scores.append(recall)
    map_scores.append(map3)

    print(f"🔍 Query: {query}")
    print(f"Expected: {expected}")
    print(f"Top Predictions: {predictions[:3]}")
    print(f"Recall@3: {recall}, MAP@3: {map3:.2f}\n")

Raw API response for 'Account Manager': [{'assessment_name': 'Account Manager', 'remote_testing': 'Yes', 'adaptive_irt': 'Yes', 'duration': '49', 'test_type': 'unknown'}]
🔍 Query: Account Manager
Expected: Account Manager
Top Predictions: ['Account Manager']
Recall@3: 1, MAP@3: 1.00

Raw API response for 'Teller': [{'assessment_name': 'Teller with Sales', 'remote_testing': 'Yes', 'adaptive_irt': 'No', 'duration': '36', 'test_type': 'unknown'}]
🔍 Query: Teller
Expected: Teller with Sales
Top Predictions: ['Teller with Sales']
Recall@3: 1, MAP@3: 1.00

Raw API response for 'Director': [{'assessment_name': 'Director', 'remote_testing': 'Yes', 'adaptive_irt': 'Yes', 'duration': '69', 'test_type': '216'}, {'assessment_name': 'District/Regional Manager', 'remote_testing': 'Yes', 'adaptive_irt': 'Yes', 'duration': '65', 'test_type': '222'}]
🔍 Query: Director
Expected: Director
Top Predictions: ['Director', 'District/Regional Manager']
Recall@3: 1, MAP@3: 1.00

Raw API response for 'Cashier': 

In [29]:
print("📊 Final Evaluation Metrics")
print(f"Mean Recall@3: {sum(recall_scores)/len(recall_scores):.2f}")
print(f"Mean MAP@3: {sum(map_scores)/len(map_scores):.2f}")

📊 Final Evaluation Metrics
Mean Recall@3: 1.00
Mean MAP@3: 0.90
